# Week 4 - Milestone 1: Smart Tracking System Blockchain Ledger (Draft)

This notebook demonstrates the initial Python integration with our `LogisticsDataStorage` smart contract deployed on a local Ganache blockchain.

**Objectives:**
1. Establish a Web3 connection to Ganache.
2. Load the deployed smart contract using its address and ABI.
3. Set the default sender account (contract owner).
4. Call `getTotalRecords()` to check the contract's initial state.
5. Store a dummy IoT data entry manually using `storeLogisticsData()`.
6. Verify the dummy data storage by retrieving the record using `getLogisticsRecord()`.

In [4]:
# Install missing dependencies
%pip install web3

# Import necessary libraries
from web3 import Web3
import json
import time # For waiting for transaction receipts

# --- Configuration ---
GANACHE_URL = "http://127.0.0.1:7545"  # Ensure this matches your Ganache RPC URL

# Contract Address (from your deployment)
CONTRACT_ADDRESS = "0xBbEde627aCF9aa814C667a46E27fb8a29227f509" 

# Sender Account (must be the contract owner for `storeLogisticsData`)
# This should be web3.eth.accounts[0] from Ganache if you used the first account for deployment
SENDER_ACCOUNT = "0xB3B75FA814041f3176d4812324CD47A0C50F31A6" 

# ABI for LogisticsDataStorage contract (as provided)
CONTRACT_ABI_JSON = """
[
	{
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"anonymous": false,
		"inputs": [
			{
				"indexed": false,
				"internalType": "uint256",
				"name": "blockchainTimestamp",
				"type": "uint256"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "originalTimestamp",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "packageId",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "rfidTag",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "latitude",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "longitude",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "temperatureC",
				"type": "string"
			},
			{
				"indexed": false,
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			}
		],
		"name": "LogisticsDataStored",
		"type": "event"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "_originalTimestamp",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_packageId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_rfidTag",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_latitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_longitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_temperatureC",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_deviceId",
				"type": "string"
			}
		],
		"name": "storeLogisticsData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getLogisticsRecord",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "blockchainTimestamp",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "originalTimestamp",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "packageId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "rfidTag",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "latitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "longitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "temperatureC",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getTotalRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "logisticsRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "blockchainTimestamp",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "originalTimestamp",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "packageId",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "rfidTag",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "latitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "longitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "temperatureC",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "MAX_ENTRIES",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "owner",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]
"""

print("Configuration loaded.")

  Using cached web3-7.11.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached eth_abi-5.2.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached eth_account-0.13.7-py3-none-any.whl.metadata (3.7 kB)
  Using cached eth_hash-0.7.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached eth_typing-5.2.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached eth_utils-5.3.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached hexbytes-1.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached aiohttp-3.11.18-cp313-cp313-win_amd64.whl.metadata (8.0 kB)
  Using cached pydantic-2.11.4-py3-none-any.whl.metadata (66 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached websockets-15.0.1-cp313-cp313-win_amd64.whl.metadata (7.0 kB)
  Using cached pyunormalize-16.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-no

In [5]:
# Connect to local Ganache blockchain
web3 = Web3(Web3.HTTPProvider(GANACHE_URL))

# Check if Python is successfully connected to the blockchain
if web3.is_connected():
    print(f"✅ Connected to Ganache at {GANACHE_URL} successfully!")
else:
    print(f"❌ Connection to {GANACHE_URL} failed. Ensure Ganache is running and the URL is correct.")
    # In a real script, you might exit here if the connection fails

✅ Connected to Ganache at http://127.0.0.1:7545 successfully!


In [6]:
# Ensure the smart contract is recognized and loaded in Python
if web3.is_connected():
    try:
        # Parse the ABI JSON string
        contract_abi = json.loads(CONTRACT_ABI_JSON)
        
        # Convert contract address to checksum format
        checksummed_contract_address = web3.to_checksum_address(CONTRACT_ADDRESS)
        
        # Load the smart contract
        contract = web3.eth.contract(address=checksummed_contract_address, abi=contract_abi)
        
        # Set the default sender address (from your specified SENDER_ACCOUNT)
        checksummed_sender_account = web3.to_checksum_address(SENDER_ACCOUNT)
        web3.eth.default_account = checksummed_sender_account
        
        print(f"✅ Smart Contract loaded. Address: {checksummed_contract_address}")
        print(f"✅ Default sender account set to: {web3.eth.default_account}")

        # Verify if the default sender is the contract owner
        contract_owner = contract.functions.owner().call()
        if web3.eth.default_account.lower() == contract_owner.lower():
            print(f"   ✅ Default sender ({web3.eth.default_account}) is the contract owner ({contract_owner}).")
        else:
            print(f"   ⚠️ WARNING: Default sender ({web3.eth.default_account}) is NOT the contract owner ({contract_owner}).")
            print("      Transactions to 'storeLogisticsData' may fail if sender is not the owner.")

    except json.JSONDecodeError:
        print("❌ Error: Could not decode ABI JSON. Please check its format.")
    except ValueError as e:
        print(f"❌ Error: Invalid address format for contract or sender. {e}")
    except Exception as e:
        print(f"❌ An error occurred while loading the contract or setting the sender: {e}")
else:
    print("Cannot load contract: Not connected to Ganache.")

✅ Smart Contract loaded. Address: 0xBbEde627aCF9aa814C667a46E27fb8a29227f509
✅ Default sender account set to: 0xB3B75FA814041f3176d4812324CD47A0C50F31A6
   ✅ Default sender (0xB3B75FA814041f3176d4812324CD47A0C50F31A6) is the contract owner (0xB3B75FA814041f3176d4812324CD47A0C50F31A6).


In [7]:
# Call getTotalRecords() to check if the contract is responding and get initial state
if 'contract' in locals() and web3.eth.default_account: # Check if contract is loaded
    print("\n--- Checking Initial Contract State ---")
    try:
        initial_total_records = contract.functions.getTotalRecords().call()
        print(f"Initial total records in contract: {initial_total_records}")
    except Exception as e:
        print(f"❌ Error calling getTotalRecords(): {e}")
        initial_total_records = -1 # Indicate an error or unknown state
else:
    print("Skipping initial record count: Contract not loaded or default account not set.")


--- Checking Initial Contract State ---
Initial total records in contract: 0


In [8]:
# Store a dummy IoT data entry manually to test if transactions work
if 'contract' in locals() and web3.eth.default_account and initial_total_records != -1:
    print("\n--- Storing Dummy IoT Data ---")
    
    # Define dummy data according to the storeLogisticsData function parameters
    dummy_original_timestamp = "2025-05-10 10:00:00"
    dummy_package_id = "PKG_DUMMY_001"
    dummy_rfid_tag = "RFID_DUMMY_TEST_A"
    dummy_latitude = "14.500000"
    dummy_longitude = "121.000000"
    dummy_temperature_c = "27.5"
    dummy_device_id = "Tracker_Dev_Dummy"

    try:
        print(f"Attempting to store dummy data for PackageID: {dummy_package_id}")
        
        # Prepare transaction details
        # The 'from' field will use web3.eth.default_account if not specified here,
        # but it's good practice to be explicit for state-changing transactions.
        transaction_details = {
            'from': web3.eth.default_account,
            'gas': 1500000  # A reasonable gas limit for this function
        }
        
        # Call the 'storeLogisticsData' function
        tx_hash = contract.functions.storeLogisticsData(
            dummy_original_timestamp,
            dummy_package_id,
            dummy_rfid_tag,
            dummy_latitude,
            dummy_longitude,
            dummy_temperature_c,
            dummy_device_id
        ).transact(transaction_details)

        print(f"  Transaction sent. Waiting for receipt... (Hash: {tx_hash.hex()})")
        
        # Wait for the transaction to be mined
        receipt = web3.eth.wait_for_transaction_receipt(tx_hash, timeout=120) # 120 seconds timeout

        if receipt.status == 1:
            print(f"✅ Dummy data for '{dummy_package_id}' stored successfully on blockchain!")
            print(f"   Transaction Hash: {receipt.transactionHash.hex()}")
            print(f"   Block Number: {receipt.blockNumber}")
            print(f"   Gas Used: {receipt.gasUsed}")
        else:
            print(f"❌ Transaction for '{dummy_package_id}' failed. Status: {receipt.status}")
            print(f"   Check Ganache logs for revert reasons if available (e.g., not owner, storage limit).")

    except Exception as e:
        print(f"❌ Error storing dummy data: {e}")
        if "revert" in str(e).lower():
            print("   This might be due to a 'require' statement in the contract (e.g., 'Not authorized', 'Storage limit reached').")
        elif "intrinsic gas too low" in str(e).lower() or "out of gas" in str(e).lower():
            print("   Try increasing the 'gas' value in the transact() call.")
else:
    print("Skipping dummy data store: Contract not loaded, default account not set, or initial checks failed.")


--- Storing Dummy IoT Data ---
Attempting to store dummy data for PackageID: PKG_DUMMY_001
  Transaction sent. Waiting for receipt... (Hash: bb7e0249416a2221b7ca675dc4cd83d45a99e317e3d8313078f5cd24994c7aef)
✅ Dummy data for 'PKG_DUMMY_001' stored successfully on blockchain!
   Transaction Hash: bb7e0249416a2221b7ca675dc4cd83d45a99e317e3d8313078f5cd24994c7aef
   Block Number: 5
   Gas Used: 243630


In [9]:
# Verify if data retrieval works
if 'contract' in locals() and web3.eth.default_account:
    print("\n--- Verifying Data Retrieval ---")
    try:
        # Get the updated total number of records
        current_total_records = contract.functions.getTotalRecords().call()
        print(f"Total Records after attempting to store dummy data: {current_total_records}")

        if current_total_records > 0 and (initial_total_records == -1 or current_total_records > initial_total_records):
            # If a new record was added, its index will be the `initial_total_records`
            # (if initial_total_records was 0 or more) or `current_total_records - 1`
            record_index_to_fetch = current_total_records - 1
            
            print(f"Attempting to retrieve the last stored record (index {record_index_to_fetch})...")
            
            # Call the 'getLogisticsRecord' function
            record = contract.functions.getLogisticsRecord(record_index_to_fetch).call()
            
            print("\nLast Stored Record Details:")
            print(f"  Blockchain Timestamp (Unix): {record[0]}")
            print(f"  Original Timestamp:          {record[1]}")
            print(f"  Package ID:                  {record[2]}")
            print(f"  RFID Tag:                    {record[3]}")
            print(f"  Latitude:                    {record[4]}")
            print(f"  Longitude:                   {record[5]}")
            print(f"  Temperature_C:               {record[6]}")
            print(f"  DeviceID:                    {record[7]}")

            # Verify if the retrieved PackageID matches the dummy PackageID
            if record[2] == "PKG_DUMMY_001":
                 print("\n✅ Verification successful: Retrieved PackageID matches the dummy data.")
            else:
                 print(f"\n⚠️ Verification issue: Retrieved PackageID '{record[2]}' does not match expected 'PKG_DUMMY_001'.")
                 print("   This could be due to other transactions or if the dummy transaction failed but other records exist.")

        elif current_total_records > 0:
            print("No new record was confirmed stored by this script's transaction, but other records exist.")
            print("Attempting to retrieve the first record (index 0) if available...")
            record_0 = contract.functions.getLogisticsRecord(0).call()
            print(f"First Stored Record (Index 0): {record_0}")
        else:
            print("No records found in the contract to retrieve.")

    except Exception as e:
        print(f"❌ Error during data retrieval: {e}")
else:
    print("Skipping data retrieval: Contract not loaded or default account not set.")


--- Verifying Data Retrieval ---
Total Records after attempting to store dummy data: 1
Attempting to retrieve the last stored record (index 0)...

Last Stored Record Details:
  Blockchain Timestamp (Unix): 1747865886
  Original Timestamp:          2025-05-10 10:00:00
  Package ID:                  PKG_DUMMY_001
  RFID Tag:                    RFID_DUMMY_TEST_A
  Latitude:                    14.500000
  Longitude:                   121.000000
  Temperature_C:               27.5
  DeviceID:                    Tracker_Dev_Dummy

✅ Verification successful: Retrieved PackageID matches the dummy data.




This notebook has successfully demonstrated the fundamental steps for Python to interact with the `LogisticsDataStorage` smart contract:
- Connection to Ganache.
- Loading the contract.
- Setting the sender account.
- Manually storing a dummy record.
- Retrieving and verifying the stored record.

The next steps would involve automating the storage of multiple records, such as those from a CSV file.

**Next Actions:**
- Save this Jupyter Notebook (`.ipynb`).
- Upload it to your GitHub repository with an appropriate commit message (e.g., "Added Milestone 1 draft: Python-blockchain basic interaction").


after manual dummy data

Now i will automate the process of reading simulated IoT data from a CSV file and storing each record on the blockchain using the smart contract. We'll limit the number of records to avoid exceeding the contract's storage cap (MAX_ENTRIES = 100).

In [10]:
import pandas as pd

# Path to the CSV file generated in Week 2
csv_path = "../week-2/simulated_logistic_iot_data.csv"

# Load the CSV data
try:
    iot_data = pd.read_csv(csv_path)
    print(f"Loaded {len(iot_data)} records from CSV.")
except Exception as e:
    print(f"❌ Error loading CSV: {e}")
    iot_data = None

Loaded 100 records from CSV.


In [11]:
# Store multiple records from the CSV to the blockchain
if iot_data is not None and 'contract' in locals() and web3.eth.default_account:
    print("\n--- Storing IoT Data from CSV to Blockchain ---")
    
    # Get current total records to avoid exceeding MAX_ENTRIES
    try:
        current_total = contract.functions.getTotalRecords().call()
        max_entries = contract.functions.MAX_ENTRIES().call()
        available_slots = max_entries - current_total
        print(f"Contract currently has {current_total} records. {available_slots} slots available.")
    except Exception as e:
        print(f"❌ Error fetching contract state: {e}")
        available_slots = 0
    
    # Limit the number of records to store
    num_to_store = min(available_slots, len(iot_data))
    print(f"Attempting to store {num_to_store} records from CSV...")
    
    for idx, row in iot_data.head(num_to_store).iterrows():
        try:
            tx_hash = contract.functions.storeLogisticsData(
                str(row['Timestamp']),
                str(row['PackageID']),
                str(row['RFIDTag']),
                str(row['Latitude']),
                str(row['Longitude']),
                str(row['Temperature_C']),
                str(row['DeviceID'])
            ).transact({
                'from': web3.eth.default_account,
                'gas': 1500000
            })
            print(f"  [{idx}] Sent tx for PackageID: {row['PackageID']} (Hash: {tx_hash.hex()})")
            # Optionally wait for each transaction to be mined (uncomment below for strict confirmation)
            # receipt = web3.eth.wait_for_transaction_receipt(tx_hash, timeout=60)
            # if receipt.status == 1:
            #     print(f"    ✅ Stored successfully.")
            # else:
            #     print(f"    ❌ Transaction failed.")
        except Exception as e:
            print(f"  [{idx}] ❌ Error storing record: {e}")
else:
    print("Skipping CSV storage: Data not loaded, contract not loaded, or default account not set.")


--- Storing IoT Data from CSV to Blockchain ---
Contract currently has 1 records. 99 slots available.
Attempting to store 99 records from CSV...
  [0] Sent tx for PackageID: PKG1001 (Hash: 44b98d5ae2a49120f59d1ff0b3f1cf3df12f4e91764f88d6d265d064debb8dc3)
  [1] Sent tx for PackageID: PKG1003 (Hash: 44bccd9bf9ff811c10fa5a332ab699b4ea3a649250ef35981b6327e2b60244e1)
  [2] Sent tx for PackageID: PKG1004 (Hash: 8a33e55716349e8cdbebeb6a98867b2e387ce27e67b6a0631af6142ad3804191)
  [3] Sent tx for PackageID: PKG1007 (Hash: d436872c75f22a935317243c0b401fc49c3222765a627f9bfa357e36e4af8d93)
  [4] Sent tx for PackageID: PKG1005 (Hash: f1a04279f9358c8b2f316fb799a28842c5a536750aafe4024d1cf112d03697b7)
  [5] Sent tx for PackageID: PKG1008 (Hash: 5cc4c157a8bcf0a99e72f1eef63f0c085a7e38efe453037fca5e975fde7996cd)
  [6] Sent tx for PackageID: PKG1008 (Hash: edd29a21096aee3c6cc5c8562eb92ca3692217847b970228d212376849acfce7)
  [7] Sent tx for PackageID: PKG1011 (Hash: 68e9d7af641e80e0ab821346b6820a241344c7a6

In [12]:
# Verify the total records after CSV automation
if 'contract' in locals() and web3.eth.default_account:
    try:
        total_after = contract.functions.getTotalRecords().call()
        print(f"\nTotal records in contract after CSV automation: {total_after}")
    except Exception as e:
        print(f"❌ Error fetching total records: {e}")


Total records in contract after CSV automation: 100
